### Use Panda to concat, delete, insert, loc the data and I/O .csv files

In [2]:
import pandas  as pd

#### 1. from tops_total.csv to operate some modifies

In [16]:
tops_df = pd.read_csv('tops_total_modified.csv')

In [17]:
#if we want to modify the row contain some errors with different names
i = tops_df[(tops_df.product_type == "cloak")].index
tops_df = tops_df.drop(i)
print(i)

Int64Index([], dtype='int64')


In [18]:
j = tops_df[((tops_df.product_type == "dress") | (tops_df.product_type == "dresses") | (tops_df.product_type == "dresses - formal"))].index
print(j)
tops_df = tops_df.drop(j)

Int64Index([], dtype='int64')


In [19]:
'''
Other operation for dataframes. 
since I put some files to different location, so the codes below just used once for creating tops_total.csv
#from all tops data we have to build the total dataset
tops_df_1 = pd.read_csv('train_data/tops.csv', usecols=["title", "tags", "product_type", "body_html"])  
tops_df_2 = pd.read_csv('sample.csv', usecols=["title", "tags", "product_type", "body_html"])
tops_df = pd.concat([tops_df_1, tops_df_2], ignore_index=True)
tops_df.to_csv('train_data/tops_total.csv')
'''

'\nOther operation for dataframes. \nsince I put some files to different location, so the codes below just used once for creating tops_total.csv\n#from all tops data we have to build the total dataset\ntops_df_1 = pd.read_csv(\'train_data/tops.csv\', usecols=["title", "tags", "product_type", "body_html"])  \ntops_df_2 = pd.read_csv(\'sample.csv\', usecols=["title", "tags", "product_type", "body_html"])\ntops_df = pd.concat([tops_df_1, tops_df_2], ignore_index=True)\ntops_df.to_csv(\'train_data/tops_total.csv\')\n'

In [24]:
k = tops_df[((tops_df.product_type == "short") | (tops_df.product_type == "shorts"))].index
print(k)
tops_df  = tops_df.drop(k)


Int64Index([], dtype='int64')


In [29]:
l = tops_df[(tops_df.product_type == "sweetlegs maternity")].index
print(l)
tops_df = tops_df.drop(l)

Int64Index([213], dtype='int64')


In [25]:
#clean all unnessary rows and columns, we output the modified tops.csv
tops_df = tops_df.loc[:, ["title",  "product_type", "tags", "body_html"]].copy()

In [26]:
tops_df.to_csv('tops.csv', index=False)

#### Create raw Train_data (.csv) and Test_data (.csv) from our total dataset (e.g 223 examples)

In [27]:
#build 180 train_data and 44 test_data 80% train 20% test for tops 223 samples
tops_df_train = tops_df.loc[0:180, ["title",  "product_type", "tags", "body_html"]].copy()
tops_df_test = tops_df.loc[180:,  ["title",  "product_type", "tags", "body_html"]].copy()
tops_df_train.to_csv('train_data/tops_train.csv', index= False)
tops_df_test.to_csv('test_data/tops_test.csv',index=False)

#### Build Customized DataSet From product.csv

In [3]:

from text_cleaner import cleanHtml
from text_cleaner import clean_product_description

columnChosen = ['title', 'product_type','tags', 'body_html']
products = pd.read_csv("/Users/luis/Downloads/products.csv", usecols=columnChosen)
products.head(10)

,title,tags,body_html,product_type
0,Aria High Waist,{},NaN,NaN
1,Hues Underwire One Piece,"{d-cup+,meta-size-chart-artesands-size-guide,m...","<meta charset=""utf-8"">\n<p><meta charset=""utf-...",one piece
2,Marble Maya One Piece,"{meta-size-chart-bound-size-guide,moderate-cov...","<meta charset=""utf-8"">\n<p data-mce-fragment=""...",NaN
3,Marble Malibu Set,"{crop,meta-size-chart-bound-size-guide,moderat...","<p data-mce-fragment=""1""><span>A Bond-Eye orig...",bikini top
4,Sophia Bottom,"{color-patterned,high-waist,meta-size-chart-po...","<meta charset=""utf-8"">\n<p>Meet your newest gi...",bikini bottom
5,"Lion 22"" Bronze",{},NaN,default
6,Sonja Top,"{color-stripes,crop,meta-size-chart-power-2-th...","<meta charset=""utf-8"">\n<p>This retro revival ...",bikini top
7,Shirley Tie-Side,"{high-waist,meta-size-chart-power-2-the-flower...","<meta charset=""utf-8"">\n<p><meta charset=""utf-...",bikini bottom
8,Gwen Ruffle Bottom,"{color-patterned,high-leg,meta-size-chart-powe...","<p class=""p1"" data-mce-fragment=""1"">Gwen Ruffl...",bikini bottom
9,Billie Bottom,"{color-patterned,meta-size-chart-power-2-the-f...","<meta charset=""utf-8"">\n<p data-mce-fragment=""...",bikini bottom


In [4]:
k = products[((products.product_type.isnull()) | (products.product_type == 'default') 
            | (products.body_html.isnull()) | (products.tags == '{}'))].index
products = products.drop(k).reset_index()

##### Drop any null and default rows in product_type and drop empty 'tags' rows

In [5]:

#pre-clean products cloumns and build our own data set columns and input-data format for NLP.
len(products.index)
r, c = products.shape[0], products.shape[1]
print(r, c)

18053 5


In [6]:
products.insert(len(products.columns), '1st_class_label', 0, allow_duplicates=True)
products.insert(len(products.columns), '2nd_class_label', 0, allow_duplicates=True)
products.insert(len(products.columns), '3rd_class_label', 0, allow_duplicates=True )
products.insert(len(products.columns), 'raw_text', "", allow_duplicates=True)


In [7]:
from text_cleaner import raw_content
for i in range(products.shape[0]):
    
    products.loc[i, 'raw_text'] = raw_content(products.loc[i, 'title'], products.loc[i, 'tags'], products.loc[i, 'body_html'])

In [8]:
products.to_csv('temp_not_finished.csv')

##### next step: 1_st_class_label, semi_automated labels.

In [9]:
products.head()

,index,title,tags,body_html,product_type,1st_class_label,2nd_class_label,3rd_class_label,raw_text
0,1,Hues Underwire One Piece,"{d-cup+,meta-size-chart-artesands-size-guide,m...","<meta charset=""utf-8"">\n<p><meta charset=""utf-...",one piece,0,0,0,Hues Underwire One Piece; d cup+ meta size cha...
1,3,Marble Malibu Set,"{crop,meta-size-chart-bound-size-guide,moderat...","<p data-mce-fragment=""1""><span>A Bond-Eye orig...",bikini top,0,0,0,Marble Malibu Set; crop meta size chart bound ...
2,4,Sophia Bottom,"{color-patterned,high-waist,meta-size-chart-po...","<meta charset=""utf-8"">\n<p>Meet your newest gi...",bikini bottom,0,0,0,Sophia Bottom; color patterned high waist meta...
3,6,Sonja Top,"{color-stripes,crop,meta-size-chart-power-2-th...","<meta charset=""utf-8"">\n<p>This retro revival ...",bikini top,0,0,0,Sonja Top; color stripes crop meta size chart ...
4,7,Shirley Tie-Side,"{high-waist,meta-size-chart-power-2-the-flower...","<meta charset=""utf-8"">\n<p><meta charset=""utf-...",bikini bottom,0,0,0,Shirley Tie Side; high waist meta size chart p...


In [10]:
set = {"bikini top"}
for i in range(products.shape[0]):
    set.add(products.loc[i, 'product_type'])
len(set)

751

In [11]:
set

{'6mm duality',
 '6mm duality collection',
 '6mm mother of pearl',
 'acc',
 'accessories',
 'accessories - scarves',
 'accessories - socks + tights',
 'accessories blanket',
 'accessories book',
 'accessories candle',
 'accessories earrings',
 'accessories fanny pack',
 'accessories furniture',
 'accessories hair care',
 'accessories hair ties/scrunchies',
 'accessories necklace',
 'accessories pillow',
 'accessories ring',
 'accessories skin care',
 'accessories suncare',
 'accessories sunglasses',
 'accessories towel',
 'accessory',
 'accesssories mugs/drinkware',
 'acessories bowl',
 'acessories coaster',
 'acessories perfumes/colgnes',
 'acessories serving tray',
 'activewear',
 'adrian klis',
 'adult hat',
 'adult top',
 'adult winterwear',
 'air freshener',
 'anklet',
 'apparel',
 'apparel and accessories',
 'arctican',
 'area rug',
 'aroma',
 'artist collection',
 'artwork',
 'athletic boxers',
 'athletics',
 'baby',
 'baby and toddler',
 'backpack',
 'backpacks',
 'bag',
 'bags